# Support Multiple Times

In [82]:
import xradar
import pyart
import glob
import dask
from distributed import Client
from datatree import DataTree
import pandas as pd
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime
xr.set_options(keep_attrs=True)

In [39]:
from datatree import open_datatree, DataTree

In [3]:
client = Client()
client

2022-07-21 14:54:52,742 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dd/zgpcpccj46q2tfrvpwpp3r5w0000gn/T/dask-worker-space/worker-xu__zgoj', purging
2022-07-21 14:54:52,743 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dd/zgpcpccj46q2tfrvpwpp3r5w0000gn/T/dask-worker-space/worker-5wx0xdif', purging
2022-07-21 14:54:52,743 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dd/zgpcpccj46q2tfrvpwpp3r5w0000gn/T/dask-worker-space/worker-8hnobssk', purging
2022-07-21 14:54:52,744 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dd/zgpcpccj46q2tfrvpwpp3r5w0000gn/T/dask-worker-space/worker-gycsqg0u', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50416,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:50437,Total threads: 3
Dashboard: http://127.0.0.1:50439/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:50422,


## Explore an Open Dataset Function

In [55]:
def open_dataset(file):
    radar = pyart.io.read(file)
    tree = xradar.convert_to_xradar(radar)
    time = create_time_from_radar(radar)
    tree["0"].ds["time"] = time
    #tree.ds["time"] = tree["0"].ds["time"].min().squeeze().drop(["fixed_angle"]).rename("start_time")
    return tree["0"].ds

def create_time_from_radar(radar):
    initial_time = pd.to_datetime(radar.time["units"][-19:])
    return pd.to_datetime(initial_time + (timedelta(seconds=1) * radar.time["data"]))

## Access Data - these datasets have been downloaded from the ARM Data Poral

In [74]:
radar1_files = sorted(glob.glob("/Volumes/Samsung_T5/argonne/sail/gucxprecipradarS2.00.20211225.231611/*4_PPI.nc"))

In [75]:
dsets = []
for file in radar1_files:
    dsets.append(open_dataset(file))

In [76]:
keys = list(map(str, list(range(len(dsets)))))

In [77]:
keys = [f"volume{s}/0" for s in keys]
keys

['volume0/0',
 'volume1/0',
 'volume2/0',
 'volume3/0',
 'volume4/0',
 'volume5/0',
 'volume6/0',
 'volume7/0',
 'volume8/0']

## Recreate our Datatree

In [78]:
dict_of_dsets = dict(zip(keys, dsets))
tree = DataTree.from_dict(dict_of_dsets)

## Add the start time for each volume

In [79]:
for child in list(tree.children):
    tree[child]["volume_start_time"] = tree[f"{child}/0"].ds["time"].min().squeeze().drop(["fixed_angle"])

Save it out to a zarr dataset

In [68]:
tree.to_zarr("test-hierarchal-tree.zarr")

---

In [87]:
from datatree import open_datatree

In [90]:
tree_opened = open_datatree("test-hierarchal-tree.zarr", engine='zarr')

In [91]:
tree_opened

DataTree('None', parent=None)
├── DataTree('volume0')
│   │   Dimensions:  ()
│   │   Data variables:
│   │       time     datetime64[ns] ...
│   └── DataTree('0')
│           Dimensions:      (azimuth: 1157, range: 675, elevation: 1157, sweep: 1,
│                             time: 1157)
│           Coordinates:
│               alt          (azimuth, range) float64 ...
│             * azimuth      (azimuth) float32 0.1251 0.1306 0.327 ... 0.2172 0.2172 0.2172
│             * elevation    (elevation) float32 3.933 3.944 3.944 ... 3.988 3.988 3.988
│               fixed_angle  float32 ...
│               lat          (azimuth, range) float32 ...
│               lon          (azimuth, range) float32 ...
│             * range        (range) float32 -112.9 -52.95 6.994 ... 4.023e+04 4.029e+04
│             * sweep        (sweep) int32 0
│             * time         (time) datetime64[ns] 2021-12-25T23:16:43 ... 2021-12-25T23:...
│               x            (azimuth, range) float32 ...
│               y            (azimuth, range) float32 ...
│               z            (azimuth, range) float32 ...
│           Data variables:
│               DBZ          (azimuth, range) float32 ...
│               DBZhv        (azimuth, range) float32 ...
│               NCP          (azimuth, range) float32 ...
│               PHIDP        (azimuth, range) float32 ...
│               RHOHV        (azimuth, range) float32 ...
│               SNR          (azimuth, range) float32 ...
│               VEL          (azimuth, range) float32 ...
│               WIDTH        (azimuth, range) float32 ...
│               ZDR          (azimuth, range) float32 ...
├── DataTree('volume1')
│   │   Dimensions:  ()
│   │   Data variables:
│   │       time     datetime64[ns] ...
│   └── DataTree('0')
│           Dimensions:      (azimuth: 1157, range: 675, elevation: 1157, sweep: 1,
│                             time: 1157)
│           Coordinates:
│               alt          (azimuth, range) float64 ...
│             * azimuth      (azimuth) float32 0.1251 0.1251 0.3161 ... 0.2172 0.2172 0.2172
│             * elevation    (elevation) float32 3.933 3.939 3.944 ... 3.988 3.988 3.988
│               fixed_angle  float32 ...
│               lat          (azimuth, range) float32 ...
│               lon          (azimuth, range) float32 ...
│             * range        (range) float32 -112.9 -52.92 7.017 ... 4.023e+04 4.029e+04
│             * sweep        (sweep) int32 0
│             * time         (time) datetime64[ns] 2021-12-25T23:22:03 ... 2021-12-25T23:...
│               x            (azimuth, range) float32 ...
│               y            (azimuth, range) float32 ...
│               z            (azimuth, range) float32 ...
│           Data variables:
│               DBZ          (azimuth, range) float32 ...
│               DBZhv        (azimuth, range) float32 ...
│               NCP          (azimuth, range) float32 ...
│               PHIDP        (azimuth, range) float32 ...
│               RHOHV        (azimuth, range) float32 ...
│               SNR          (azimuth, range) float32 ...
│               VEL          (azimuth, range) float32 ...
│               WIDTH        (azimuth, range) float32 ...
│               ZDR          (azimuth, range) float32 ...
├── DataTree('volume2')
│   │   Dimensions:  ()
│   │   Data variables:
│   │       time     datetime64[ns] ...
│   └── DataTree('0')
│           Dimensions:      (azimuth: 1156, range: 675, elevation: 1156, sweep: 1,
│                             time: 1156)
│           Coordinates:
│               alt          (azimuth, range) float64 ...
│             * azimuth      (azimuth) float32 0.1251 0.1251 0.3215 ... 0.2172 0.2172 0.2172
│             * elevation    (elevation) float32 3.933 3.933 3.944 ... 3.988 3.988 3.988
│               fixed_angle  float32 ...
│               lat          (azimuth, range) float32 ...
│               lon          (azimuth, range) float32 ...
│             * range  